I want to download the transcript of this youtube video :
https://www.youtube.com/watch?v=VTmwhwu1JEc
Write a python script using this library: youtube-transcript-api

Then, connect to an LLM programatically to generate a summary of the video.


Okay, I need to write a Python script that uses the youtube-transcript-api to get a transcript and then connect to an LLM to generate a summary. Let me start by breaking down the problem.

First, the user wants the transcript of a specific YouTube video. The URL given is https://www.youtube.com/watch?v=VTmwhwu1JEc. I remember that the youtube-transcript-api allows fetching transcripts by video ID. So I'll need to extract the video ID from the URL. The video ID is the part after v=, which here is VTmwhwu1JEc.

Next, I'll use the YouTubeTranscriptApi to get the transcript. But wait, sometimes transcripts might not be available, so I should include a try-except block to handle exceptions. The get_transcript function returns a list of dictionaries with 'text', 'start', and 'duration' keys. I need to concatenate all the 'text' parts to form the full transcript.

Once I have the transcript, the next step is to connect to an LLM. The user didn't specify which LLM, but common choices are OpenAI's GPT or maybe an open-source model via an API. Since OpenAI is widely used and has a straightforward API, I'll go with that. But I should mention alternative options in case the user prefers another service.

For OpenAI, I need to import the OpenAI library and set up the client with an API key. The user will need to have an API key, so I'll include a reminder to install the library and set the environment variable.

Now, to generate the summary. The transcript might be long, so I have to consider the token limit. If the transcript is too long, sending it all at once might exceed the model's maximum context window. To handle this, I can split the text into chunks, summarize each chunk, and then combine the summaries. But for simplicity, maybe the example assumes the transcript is short enough. However, I should note this limitation and suggest handling chunking if necessary.

Putting it all together, the steps are:
1. Install necessary libraries: youtube-transcript-api and openai.
2. Fetch the transcript using the video ID.
3. Combine the text entries into a single string.
4. Use the OpenAI client to send a prompt asking for a summary.
5. Print the summary.

Wait, but the user mentions connecting to an LLM programmatically. If they don't want to use OpenAI, perhaps they can use another service. But since the example uses OpenAI, I'll proceed with that. I'll make sure to include error handling for both the transcript fetch and the API call.

In the script, after getting the transcript, I'll check if it's not empty. Then, create a prompt like "Please summarize the following video transcript:\n\n[transcript text]...". Send this to the LLM and get the response.

Possible issues: The video might not have a transcript available, or it might be in a different language. The youtube-transcript-api can handle languages, but by default, it tries English. The user might need to specify a language if needed. Also, the OpenAI API key needs to be set, which I'll remind them about.

Testing the script: After installing the packages, run the script with the video ID. Ensure the API key is set in the environment. Handle any exceptions that occur during API calls, like network errors or invalid keys.

So the final script will have two main parts: fetching the transcript and generating the summary. Let me structure the code with comments and error handling. Also, mention alternatives like using local models with Llama.cpp or HuggingFace's API if the user prefers.


Here's a Python script that fetches the YouTube transcript and generates a summary using an LLM (using OpenAI as an example):

```python
# First install required packages:
# pip install youtube-transcript-api openai

from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
import os

def get_youtube_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_text = " ".join([entry['text'] for entry in transcript])
        return full_text
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None

def generate_summary(text, model="gpt-3.5-turbo"):
    # Initialize OpenAI client - get your API key from https://platform.openai.com/
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes video transcripts."},
                {"role": "user", "content": f"Please summarize the key points of this video transcript in 3-5 concise bullet points:\n\n{text}"}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating summary: {e}")
        return None

if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=VTmwhwu1JEc"
    video_id = video_url.split("v=")[-1]
    
    print(f"Fetching transcript for video: {video_id}")
    transcript = get_youtube_transcript(video_id)
    
    if transcript:
        print("\nGenerating summary...")
        summary = generate_summary(transcript)
        if summary:
            print("\nSummary:")
            print(summary)
        else:
            print("Failed to generate summary")
    else:
        print("Could not retrieve transcript")
```

To use this script:

1. Install required packages:
```bash
pip install youtube-transcript-api openai
```

2. Set your OpenAI API key as environment variable:
```bash
export OPENAI_API_KEY='your-api-key-here'
```

3. Run the script

Key notes:
- The script extracts the video ID from the URL automatically
- Handles potential errors in transcript fetching
- Uses OpenAI GPT-3.5-turbo by default (good balance of cost/performance)
- The summary prompt asks for bullet points, but you can modify this by changing the user message content
- Works best with videos that have proper captions available

Alternative LLM options (replace `generate_summary` function):
1. For open-source models (using HuggingFace):
   ```python
   from transformers import pipeline
   def generate_summary(text):
       summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
       return summarizer(text, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
   ```

2. For Anthropic Claude:
   ```python
   from anthropic import Anthropic
   def generate_summary(text):
       client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
       response = client.messages.create(
           model="claude-3-opus-20240229",
           max_tokens=400,
           messages=[{
               "role": "user",
               "content": f"Summarize this video transcript in 5 bullet points:\n\n{text}"
           }]
       )
       return response.content[0].text
   ```

Remember to handle:
- Long transcripts (you might need to truncate or split text if exceeding model context window)
- Different languages (add language parameter to YouTubeTranscriptApi.get_transcript)
- API rate limits and error handling for production use


In [ ]:
from openai import OpenAI


def generate_summary(text, model="gpt-3.5-turbo"):
    # Initialize OpenAI client - get your API key from https://platform.openai.com/
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that summarizes video transcripts.",
                },
                {
                    "role": "user",
                    "content": f"Please summarize the key points of this video transcript in 3-5 concise bullet points:\n\n{text}",
                },
            ],
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating summary: {e}")
        return None

In [ ]:
# First install required packages:
# pip install youtube-transcript-api openai

from youtube_transcript_api import YouTubeTranscriptApi
import os

def get_youtube_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_text = " ".join([entry['text'] for entry in transcript])
        return full_text
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None



if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=VTmwhwu1JEc"
    video_id = video_url.split("v=")[-1]
    
    print(f"Fetching transcript for video: {video_id}")
    transcript = get_youtube_transcript(video_id)
    
    if transcript:
        print("\nGenerating summary...")
        summary = generate_summary(transcript)
        if summary:
            print("\nSummary:")
            print(summary)
        else:
            print("Failed to generate summary")
    else:
        print("Could not retrieve transcript")

In [ ]:
# Install required packages:
# pip install youtube-transcript-api openai tiktoken tenacity

from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
import os
import tiktoken
from tenacity import retry, wait_exponential, stop_after_attempt
import re

# Configurations
MAX_TOKENS = {
    "gpt-3.5-turbo": 16385,
    "gpt-4": 128000
}
DEFAULT_MODEL = "gpt-3.5-turbo"
TARGET_LANGUAGE = "en"  # ISO 639-1 language code
SUMMARY_INSTRUCTION = "Summarize in 3-5 concise bullet points in {language}:"

def get_youtube_transcript(video_id, languages=[TARGET_LANGUAGE]):
    try:
        # First try preferred language
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)
        except:
            # Fallback to available transcripts if preferred language not available
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            
        full_text = " ".join([entry['text'] for entry in transcript])
        return full_text
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None

def chunk_text(text, model_name=DEFAULT_MODEL, max_tokens=None):
    if max_tokens is None:
        max_tokens = MAX_TOKENS.get(model_name, 16000) // 2  # Conservative buffer
        
    encoder = tiktoken.encoding_for_model(model_name)
    tokens = encoder.encode(text)
    
    chunks = []
    current_chunk = []
    current_length = 0
    
    for token in tokens:
        if current_length + 1 > max_tokens:
            chunks.append(encoder.decode(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(token)
        current_length += 1
    
    if current_chunk:
        chunks.append(encoder.decode(current_chunk))
    
    return chunks

@retry(wait=wait_exponential(multiplier=1, min=4, max=60), 
       stop=stop_after_attempt(3),
       reraise=True)
def generate_summary(text, model=DEFAULT_MODEL):
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"), timeout=30)
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system", 
                    "content": "You are a professional summarizer. Focus on key insights and main points."
                },
                {
                    "role": "user",
                    "content": SUMMARY_INSTRUCTION.format(language=TARGET_LANGUAGE) + f"\n\n{text}"
                }
            ],
            temperature=0.3
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"API Error: {type(e).__name__} - {str(e)}")
        raise

def process_long_transcript(full_text, model=DEFAULT_MODEL):
    chunks = chunk_text(full_text, model)
    
    if len(chunks) == 1:
        return generate_summary(full_text, model)
        
    summaries = []
    for chunk in chunks:
        summary = generate_summary(chunk, model)
        summaries.append(summary)
    
    combined = "\n\n".join(summaries)
    return generate_summary(combined, model)

def clean_summary(summary):
    # Remove markdown formatting if present
    return re.sub(r"[\*\-•]+", "", summary).strip()

if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=VTmwhwu1JEc"
    video_id = video_url.split("v=")[-1]
    
    print(f"Fetching transcript for video: {video_id}")
    transcript = get_youtube_transcript(video_id, languages=["en", "fr", "es"])
    
    if transcript:
        print("\nProcessing transcript...")
        try:
            final_summary = process_long_transcript(transcript)
            clean_output = clean_summary(final_summary)
            print("\nSummary:")
            print(clean_output)
        except Exception as e:
            print(f"Failed to generate summary: {str(e)}")
    else:
        print("Could not retrieve transcript")



Okay, let's tackle this. The user wants a Python script that uses an open-access LLM API without requiring tokens or accounts. So, first, I need to think about which LLMs are available for free without any API keys.

Hmm, the Hugging Face Inference API has some free tiers, but it usually requires an API token. Maybe the Free APIs that don't need authentication? I remember that some models hosted on Hugging Face Spaces can be accessed without tokens, but they might have rate limits. For example, the T5-Summarization model on a public Space could work.

Alternatively, maybe use a local model with the transformers library. But that would require the user to download the model, which might not be ideal if they want something quick without setup. However, the user specifically asked for an API, so local models are out. So maybe find a public endpoint.

Wait, the Databricks Dolly model has a demo API endpoint that doesn't need a token. Let me check. Yes, the endpoint at https://api-inference.huggingface.co/models/databricks/dolly-v2-12b could work. But wait, does that require a token? Because most Hugging Face inference endpoints do. Oh, right, usually you need a token even for public models. So maybe that's not feasible.

Alternatively, maybe use the Hugging Face Inference API for a public model. For example, the Facebook's BART model for summarization. But again, that requires an API token. The user wants no tokens or accounts. So maybe that's not possible through official APIs.

Wait, perhaps some models are hosted on public Spaces with exposed API endpoints that don't require authentication. For example, someone might have deployed a summarization model on a Space and allowed public access via the Gradio API. Let me think. For instance, using the Gradio client to connect to a public Space. That could bypass the need for API tokens.

Yes! Some Hugging Face Spaces have Gradio interfaces, and the Gradio client can be used to interact with them programmatically without tokens. So, for example, a Space that takes text input and returns a summary could be accessed via its public API.

Let me find an example of such a Space. Suppose there's a Space called "facebook-bart-large-cnn-summarization" or similar. Let me check. Hmm, maybe not exact, but there are Spaces with public APIs. For instance, the "philschmid/bart-large-cnn-samsum" Space. But I need to verify if that's accessible via API without a token.

Alternatively, maybe use the Hugging Face's model inference API without a token. But according to their docs, you need an API token for inference endpoints. So that's not an option.

Wait, the user might not mind using an unofficial endpoint, but I should be cautious about reliability. Alternatively, perhaps use a different service altogether, like DeepAI's free tier, but that requires an API key. So not suitable.

Alternatively, use a locally run model with the transformers library. But that requires the user to download the model, which can be large. But the user might accept that if it's the only way. For example, using the pipeline with a small model like "sshleifer/distilbart-cnn-12-6".

But the user explicitly asked for an API. So perhaps another approach. Maybe use the anonymous access to the Hugging Face Inference API. Wait, no, I don't think that's allowed. Let me check the API documentation. From what I remember, you need to have a token for the Inference API, even for public models.

Hmm. This is a problem. The user doesn't want any tokens or accounts. So what's a truly open-access API? Maybe some alternative services. Let me think. Perhaps the ChatGPT demo, but that's not allowed. Or other open-source models hosted somewhere.

Wait, maybe the public LLM APIs provided by universities or research groups. For example, the Allen Institute's models, but I'm not sure. Alternatively, maybe use the GPT4All API, but that's local as well.

Alternatively, use a free web service that has an API without authentication. But I can't think of any reliable ones. This seems challenging.

Wait, going back to the user's original question: they want to use the youtube-transcript-api and an open-access LLM API without tokens or accounts. So perhaps the answer is to use a local model via the transformers library, even though it's not an API. The user might have misunderstood that APIs usually require tokens, so suggesting a local model could be the way to go.

So the assistant's last message proposed using Hugging Face models via the pipeline, which runs locally. That fits the "no token nor account needed" but requires downloading the model. Let me check the previous answers. The user previously asked for a Hugging Face alternative, and the assistant provided a script using the pipeline with the transformers library. So maybe the user wants something similar but even more accessible.

Therefore, the correct approach is to use a local summarization model with the transformers library, which requires no API tokens or accounts. The assistant can create a script using, say, the small BART model for summarization, which can be run entirely locally, without any API calls.

Yes, that makes sense. So the script would use youtube-transcript-api to get the transcript and then use the pipeline from the transformers library to summarize it. The user will need to install the necessary packages and download the model, but no accounts or tokens are required.

Now, considering the model size, maybe suggest a smaller model for faster inference. Models like 'sshleifer/distilbart-cnn-12-6' are smaller and efficient. However, the summarization quality might be lower than larger models, but it's acceptable for a free solution.

Additionally, handle long transcripts by chunking the text and summarizing each chunk before combining. Also, add error handling and language options.

So putting it all together, the script would look something like:

1. Fetch the transcript using youtube-transcript-api.
2. Split the transcript into chunks if it's too long for the model context.
3. Use the transformers pipeline to summarize each chunk.
4. Combine the summaries and generate a final summary if necessary.
5. Handle different languages by specifying the language parameter when fetching the transcript.

The script can use the pipeline with a model that doesn't require an API key, runs locally, and doesn't need any user accounts. That meets the user's requirements.


Here's a solution using a local LLM through the `transformers` library with a small model that can run without API keys or external services:

```python
# Install required packages first:
# pip install youtube-transcript-api transformers torch sentencepiece

from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline
import re

MODEL_NAME = "Falconsai/text_summarization"  # Small 600MB model good for summarization

def get_transcript(video_id, language='en'):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])
        return " ".join([entry['text'] for entry in transcript])
    except:
        try:
            # Fallback to any available transcript
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            return " ".join([entry['text'] for entry in transcript])
        except Exception as e:
            print(f"Transcript error: {str(e)}")
            return None

def chunk_text(text, max_length=512):
    """Split text into chunks without breaking sentences"""
    sentences = re.split(r'(? max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(sentence)
        current_length += sentence_length
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

def summarize_local(text, model_name=MODEL_NAME):
    summarizer = pipeline(
        "summarization",
        model=model_name,
        tokenizer=model_name
    )
    
    chunks = chunk_text(text)
    summaries = []
    
    for chunk in chunks:
        summary = summarizer(
            chunk,
            max_length=150,
            min_length=30,
            do_sample=False
        )[0]['summary_text']
        summaries.append(summary)
    
    # Combine and summarize final results if needed
    if len(summaries) > 1:
        combined = " ".join(summaries)
        return summarizer(
            combined,
            max_length=200,
            min_length=50,
            do_sample=False
        )[0]['summary_text']
    
    return summaries[0]

if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=VTmwhwu1JEc"
    video_id = video_url.split("v=")[-1].split("&")[0]
    
    print(f"Fetching transcript for: {video_id}")
    transcript = get_transcript(video_id)
    
    if transcript:
        print("Generating summary (this may take a while)...")
        try:
            summary = summarize_local(transcript)
            print("\nSummary:")
            print(summary)
        except Exception as e:
            print(f"Summarization failed: {str(e)}")
    else:
        print("Could not retrieve transcript")
```

Key features:
1. **No API keys/accounts required** - Runs completely locally
2. **Handles long transcripts** - Splits text into logical chunks
3. **Multi-language support** - Change the `language` parameter in `get_transcript()`
4. **Small footprint** - Model is only ~600MB

Alternative models you can try by changing `MODEL_NAME`:
- `sshleifer/distilbart-cnn-12-6` (1.6GB, better quality)
- `philschmid/bart-large-cnn-samsum` (1.5GB)
- `lxyuan/distilbart-finetuned-financial-summarization` (for financial content)

To use:
```bash
pip install youtube-transcript-api transformers torch sentencepiece
python script.py
```

Note:
- The first run will download the model (600MB-2GB depending on model)
- Processing time depends on your CPU/GPU (works on CPU but faster with GPU)
- Quality varies based on model size - larger models give better results

For better performance:
1. Add this before creating the pipeline if you have a GPU:
```python
import torch
device = 0 if torch.cuda.is_available() else -1
summarizer = pipeline(..., device=device)
```
2. Use a quantized version of the model (if available)
3. Limit the input video length (under 15 minutes is ideal)

This is a true open-access solution that works entirely locally without any external services or authentication requirements.

---
undefined: https://labs.perplexity.ai/?utm_source=copy_output

In [ ]:
# Install required packages:
# pip install youtube-transcript-api openai tiktoken tenacity

from youtube_transcript_api import YouTubeTranscriptApi
import os
import re

TARGET_LANGUAGE = "en"  # ISO 639-1 language code
SUMMARY_INSTRUCTION = "Summarize in 3-5 concise bullet points in {language}:"


def get_youtube_transcript(video_id, languages=(TARGET_LANGUAGE,)):
    try:
        # First try preferred language
        try:
            transcript = YouTubeTranscriptApi.get_transcript(
                video_id, languages=languages
            )
        except:
            # Fallback to available transcripts if preferred language not available
            transcript = YouTubeTranscriptApi.get_transcript(video_id)

        full_text = " ".join([entry["text"] for entry in transcript])
        return full_text
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None


if __name__ == "__main__":
    # La Chine "encercle" Taïwan : une démonstration de force pour tester Donald Trump ?
    video_url = "https://www.youtube.com/watch?v=VTmwhwu1JEc"
    video_id = video_url.split("v=")[-1]

    print(f"Fetching transcript for video: {video_id}")
    transcript = get_youtube_transcript(video_id, languages=["en", "fr", "es"])

    if transcript:
        print(transcript)
    else:
        print("Could not retrieve transcript")

In [ ]:
print(transcript)

In [ ]:
word_count = (len(transcript.split(" ")))
print(word_count)

In [ ]:
import itertools

for batch in itertools.batched(transcript.split(" "), n=8):
    print(" ".join(batch))

In [ ]:
import itertools 

for batch in itertools.batched(transcript, n=80):
    print("".join(batch))

In [ ]:
prompt = f"""
Tu es un assistant qui synthétises les transcripts vidéo. 
S'il-te-plait, résume les points clefs du transcript qui suit. 
Le transcript est en français et je souhaite un résumé en français.

{transcript}

"""

In [ ]:
print(prompt)


Okay, so I need to summarize the key points of this French transcript. Let me start by reading through it carefully to understand what's going on. 

The transcript is about a tense radio exchange between a Taiwanese coast guard and a Chinese ship. China has deployed military vessels around Taiwan, conducting naval and aerial maneuvers as a show of force. There's talk of a simulated blockade and precision strikes, which the Chinese media is broadcasting as propaganda. The Taiwanese president is being called a "parasite" and threatened with destruction. Taiwan condemns these intimidations, suggesting China is using "gray zone" tactics—harassment and propaganda to pressure Taiwan into submission. China is escalating pressure, warning that anti-independence measures will continue as long as Taiwan seeks independence.

This comes after the U.S. Secretary of Defense visited the region, reaffirming support for Taiwan. The U.S. emphasizes a credible deterrent in the Indo-Pacific, including the Taiwan Strait. The discussion then moves to analysts debating the risk of a Chinese invasion. Marc Julien from IFRI argues that China's military exercises are not just shows of force but preparations for potential operations. Alice Seman, a sinologist, thinks there's no immediate invasion risk; these maneuvers are more about testing U.S. responses under Trump's administration. Frédéric Ansel, a geopolitical analyst, points out China's caution due to economic dependencies, especially on Taiwanese semiconductors, and the lessons from Russia's struggles in Ukraine.

They also discuss the historical context, like China's missile tests in 1996 and the U.S. response with carrier groups. Trump's transactional approach to alliances and Taiwan's strategic importance in semiconductor production (TSMC investing in the U.S.) are noted. The analysts debate whether the U.S. under Trump will maintain support for Taiwan, given his focus on economic gains and recent tariffs. The conversation touches on China-Russia relations, their coordinated strategies against the U.S., and China's long-term ambition to become a global superpower by 2049, despite current economic challenges.

Now, I need to distill this into a concise summary in French. Key elements to include: Chinese military maneuvers as intimidation, Taiwan and U.S. responses, analysts' views on invasion risks, economic factors (semiconductors), Trump's policies, and Sino-Russian relations. Make sure the summary flows logically, highlighting the main points without getting bogged down in every detail. Use clear sections if needed but keep it a coherent paragraph. Check for any nuances in the analysts' opinions and ensure they're accurately represented. Also, note the mention of historical context (1996) and future ambitions (2049). Avoid jargon but keep technical terms where necessary (like "stratégie de la zone grise"). Finally, verify all names and titles for correctness.


**Résumé des points clés du transcript :**

1. **Manœuvres militaires chinoises autour de Taïwan** :  
   La Chine a déployé des navires et avions militaires, simulant un blocus et des frappes ciblées sur Taïwan, accompagnés d'une propagande affirmant sa souveraineté sur l'île. Ces exercices interarmées visent à tester l'interopérabilité des forces chinoises et à intimider Taïwan, qualifiant son président de "parasite" et menaçant une "destruction totale".

2. **Réactions internationales** :  
   - **Taïwan** condamne ces actions comme une stratégie de "zone grise" (harcèlement et propagande) pour affaiblir sa résistance.  
   - **Les États-Unis** réaffirment leur soutien via le secrétaire à la Défense Lloyd Austin, promettant une dissuasion crédible dans le détroit de Taïwan, bien que l'administration Trump adopte une approche mercantile et transactionnelle des alliances.

3. **Analyse des risques d'invasion** :  
   - **Alice Seman** estime qu'une invasion n'est pas immédiate : les manœuvres servent surtout à tester la réaction de l'administration Trump, dans un contexte de rivalité sino-américaine durable.  
   - **Marc Julien** souligne la préparation croissante de l'armée chinoise, évoquant une fenêtre d'opportunité stratégique (objectif de 2027, centenaire de l'Armée populaire).  
   - **Frédéric Ansel** met en avant les risques économiques (dépendance aux semi-conducteurs taïwanais) et les leçons de la guerre en Ukraine, incitant Pékin à la prudence.

4. **Enjeux économiques et technologiques** :  
   Taïwan, producteur de 90% des semi-conducteurs de pointe, renforce ses investissements aux États-Unis (TSMC) pour sécuriser son alliance. Trump impose des droits de douane élevés, mais épargne temporairement les puces taïwanaises, reflétant leur importance stratégique.

5. **Contexte historique et ambitions chinoises** :  
   Les exercices rappellent la crise de 1996, où les États-Unis avaient déployé des porte-avions. Pékin poursuit son ambition d'être une superpuissance d'ici 2049, malgré des défis économiques structurels. La Chine renforce aussi son "amitié" avec la Russie (visites régulières, coordination géopolitique), cherchant à contrer l'influence occidentale.

6. **Relations Sino-Russes** :  
   Le rapprochement avec Moscou, continu depuis 2014, sert à coordonner une opposition commune aux États-Unis. Cependant, des tensions subsistent, notamment face aux tentatives de Trump de se rapprocher de Poutine.

**Conclusion** : Si une invasion immédiate de Taïwan semble improbable, les manœuvres chinoises illustrent une escalade calculée pour tester les réponses américaines et affirmer la puissance régionale de Pékin, dans un équilibre complexe entre intimidation, dépendances économiques et ambitions géopolitiques à long terme.

---
undefined: https://labs.perplexity.ai/?utm_source=copy_output

NOMS CORRIGES 


Okay, so I need to summarize the key points of this French transcript. Let me start by reading through it carefully to understand what's going on. 

The transcript is about a tense radio exchange between a Taiwanese coast guard and a Chinese ship. China has deployed military vessels around Taiwan, conducting naval and aerial maneuvers as a show of force. There's talk of a simulated blockade and precision strikes, which the Chinese media is broadcasting as propaganda. The Taiwanese president is being called a "parasite" and threatened with destruction. Taiwan condemns these intimidations, suggesting China is using "gray zone" tactics—harassment and propaganda to pressure Taiwan into submission. China is escalating pressure, warning that anti-independence measures will continue as long as Taiwan seeks independence.

This comes after the U.S. Secretary of Defense visited the region, reaffirming support for Taiwan. The U.S. emphasizes a credible deterrent in the Indo-Pacific, including the Taiwan Strait. The discussion then moves to analysts debating the risk of a Chinese invasion. Marc Julien from IFRI argues that China's military exercises are not just shows of force but preparations for potential operations. Alice Seman, a sinologist, thinks there's no immediate invasion risk; these maneuvers are more about testing U.S. responses under Trump's administration. Frédéric Ansel, a geopolitical analyst, points out China's caution due to economic dependencies, especially on Taiwanese semiconductors, and the lessons from Russia's struggles in Ukraine.

They also discuss the historical context, like China's missile tests in 1996 and the U.S. response with carrier groups. Trump's transactional approach to alliances and Taiwan's strategic importance in semiconductor production (TSMC investing in the U.S.) are noted. The analysts debate whether the U.S. under Trump will maintain support for Taiwan, given his focus on economic gains and recent tariffs. The conversation touches on China-Russia relations, their coordinated strategies against the U.S., and China's long-term ambition to become a global superpower by 2049, despite current economic challenges.

Now, I need to distill this into a concise summary in French. Key elements to include: Chinese military maneuvers as intimidation, Taiwan and U.S. responses, analysts' views on invasion risks, economic factors (semiconductors), Trump's policies, and Sino-Russian relations. Make sure the summary flows logically, highlighting the main points without getting bogged down in every detail. Use clear sections if needed but keep it a coherent paragraph. Check for any nuances in the analysts' opinions and ensure they're accurately represented. Also, note the mention of historical context (1996) and future ambitions (2049). Avoid jargon but keep technical terms where necessary (like "stratégie de la zone grise"). Finally, verify all names and titles for correctness.


*Résumé des points clés du transcript :*

1. *Manœuvres militaires chinoises autour de Taïwan* :  
   La Chine a déployé des navires et avions militaires, simulant un blocus et des frappes ciblées sur Taïwan, accompagnés d'une propagande affirmant sa souveraineté sur l'île. Ces exercices interarmées visent à tester l'interopérabilité des forces chinoises et à intimider Taïwan, qualifiant son président de "parasite" et menaçant une "destruction totale".

2. *Réactions internationales* :  
   - *Taïwan* condamne ces actions comme une stratégie de "zone grise" (harcèlement et propagande) pour affaiblir sa résistance.  
   - *Les États-Unis* réaffirment leur soutien via le secrétaire à la Défense Lloyd Austin, promettant une dissuasion crédible dans le détroit de Taïwan, bien que l'administration Trump adopte une approche mercantile et transactionnelle des alliances.

3. *Analyse des risques d'invasion* :  
   - *Alice Ekman* estime qu'une invasion n'est pas immédiate : les manœuvres servent surtout à tester la réaction de l'administration Trump, dans un contexte de rivalité sino-américaine durable.  
   - *Marc Julienne* souligne la préparation croissante de l'armée chinoise, évoquant une fenêtre d'opportunité stratégique (objectif de 2027, centenaire de l'Armée populaire).  
   - *Frédéric Encel* met en avant les risques économiques (dépendance aux semi-conducteurs taïwanais) et les leçons de la guerre en Ukraine, incitant Pékin à la prudence.

4. *Enjeux économiques et technologiques* :  
   Taïwan, producteur de 90% des semi-conducteurs de pointe, renforce ses investissements aux États-Unis (TSMC) pour sécuriser son alliance. Trump impose des droits de douane élevés, mais épargne temporairement les puces taïwanaises, reflétant leur importance stratégique.

5. *Contexte historique et ambitions chinoises* :  
   Les exercices rappellent la crise de 1996, où les États-Unis avaient déployé des porte-avions. Pékin poursuit son ambition d'être une superpuissance d'ici 2049, malgré des défis économiques structurels. La Chine renforce aussi son "amitié" avec la Russie (visites régulières, coordination géopolitique), cherchant à contrer l'influence occidentale.

6. *Relations Sino-Russes* :  
   Le rapprochement avec Moscou, continu depuis 2014, sert à coordonner une opposition commune aux États-Unis. Cependant, des tensions subsistent, notamment face aux tentatives de Trump de se rapprocher de Poutine.

*Conclusion* : Si une invasion immédiate de Taïwan semble improbable, les manœuvres chinoises illustrent une escalade calculée pour tester les réponses américaines et affirmer la puissance régionale de Pékin, dans un équilibre complexe entre intimidation, dépendances économiques et ambitions géopolitiques à long terme.

In [ ]:
# La Chine "encercle" Taïwan : une démonstration de force pour tester Donald Trump ?
video_url = "https://www.youtube.com/watch?v=VTmwhwu1JEc"

# Le Pen, Sarkozy, etc. : peut-on juger les politiques sans faire de politique ? | 28 minutes | ARTE
video_url = "https://www.youtube.com/watch?v=6JGOZ6KzySo"

# La CHUTE ANNONCÉE de la maison Le Pen
video_url = "https://www.youtube.com/watch?v=blaMIWIIGsc"

# Redéfinition pénale du viol, inéligibilité de Le Pen, Grok : Le Club | 28 minutes | ARTE
video_url = "https://www.youtube.com/watch?v=zth-w2tHv8Q"
video_title = "Redéfinition pénale du viol, inéligibilité de Le Pen, Grok : Le Club | 28 minutes | ARTE"
video_description = """
Ce vendredi, Jean-Mathieu Pernin décrypte l’actualité avec le regard de nos clubistes : Pascal Blanchard, historien, Blanche Leridon, directrice éditoriale de l'Institut Montaigne, Aziliz Le Corre, rédactrice en chef "Opinions" au JDD, et le dessinateur Thibaut Soulcié.  

Au programme : une nouvelle définition pénale du viol, les réactions à la guerre commerciale de Trump, les débats autour de l'inégibilité à effet immédiat, une nouvelle tendance sur les réseaux sociaux qui se moque des Français, l’IA Grok se retourne contre Elon Musk et pour qui roulent les valises à roulettes ? 

Constance Lagrange est autrice et illustratrice de BD et publie "On peut rire de tout (sauf de sa mère)" aux éditions Dargaud. Elle  consacre ses planches à ses origines juives, au travers d'histoires humoristiques mettant en scène des enfants, des vieux couples, des rabbins, des belles-mères, etc. Un humour typiquement juif qui est le thème central de cet ouvrage car "les juifs aiment rire, et d'abord d'eux-mêmes", préface l'historien Ivan Jablonka.


Sommaire
Le consentement inscrit dans la définition pénale du viol : une avancée majeure ?
Keir Starmer vs Anthony Albanese : comment réagir à la guerre commerciale de Trump ?
Chine contre Taïwan : l’autre guerre qui nous guette ?
Après le procès Le Pen, faut-il supprimer l’inéligibilité immédiate ?
“I don’t wanna be French” : se moquer de la France, c’est tendance !
“On peut rire de tout (sauf de sa mère)” : pourquoi l’humour juif est indispensable
Elon Musk, l’arroseur arrosé par sa propre intelligence artificielle
Les photos de la semaine
Pour qui roulent les valises à roulettes ? 


#Viol #MarineLePen #Inéligibilité


28 minutes est le magazine d’actualité d’ARTE, présenté par Élisabeth Quin du lundi au jeudi, à 20h05. Renaud Dély est aux commandes de l'émission le vendredi et le samedi. Retrouvez toutes les informations sur notre site : http://www.arte.tv/28minutes.
"""

video_id = video_url.split("v=")[-1]
print(f"Fetching transcript for video: {video_id}")
transcript = get_youtube_transcript(video_id, languages=["en", "fr", "es"])

if transcript:
    print(transcript)
else:
    print("Could not retrieve transcript")

# Couvre la diversité des opinions.

# Prompt with metadata
prompt = f"""
Tu es un assistant qui synthétises les transcripts vidéo. 
S'il-te-plait, résume les points clefs du transcript qui suit. 
Le transcript est en français et je souhaite un résumé en français.

Pour le contexte, voici le titre de la video : {video_title}

Pour le contexte, voici la description de la video : 

{video_description}

Enfin, voici le transcript:

{transcript}
"""

# basic prompt
prompt = f"""
Tu es un assistant qui synthétises les transcripts vidéo. 
S'il-te-plait, résume les points clefs du transcript qui suit. 
Le transcript est en français et je souhaite un résumé en français.
Voici le transcript:

{transcript}
"""
display("")
print(prompt)


Okay, I need to summarize the key points of this French video transcript about Marine Le Pen's legal issues and the political implications. Let me start by reading through the transcript again to make sure I understand all the main elements.

First, the transcript discusses the condemnation of Marine Le Pen to five years of ineligibility, which affects her potential candidacy for the 2027 presidential election. The National Rally party is accusing the judiciary of political interference, specifically targeting the judge who issued the ruling. There's mention of the judge receiving police protection due to threats, which is significant.

Then, there's a historical context provided where both left and right politicians have accused the judiciary of bias when facing legal troubles. Examples include François Fillon in 2017 and Nicolas Sarkozy in the Libyan financing case. The debate in the transcript revolves around whether the judiciary is truly politicized or if these accusations are a defense strategy by politicians.

Experts on the panel discuss the independence of judges and the balance between criticizing judicial decisions and undermining the judiciary's credibility. They mention that while judges can be criticized through legal channels, there's a line where criticism becomes harmful to the institution.

Another point is the comparison of political figures from different sides who have been convicted, suggesting that the issue isn't one-sided but part of a broader challenge to liberal democracy. The discussion also touches on the provisional execution of sentences, which immediately applies the ineligibility penalty, and whether the law should be changed to prevent such immediate effects, especially during election periods.

Magistrates and commentators argue about the need for judicial reform versus maintaining the rule of law without political interference. The tension between legal decisions and their political consequences is a recurring theme, with calls for political probity and respect for judicial processes.

Finally, there's a debate on whether the expedited appeal process for Marine Le Pen is a special treatment or a necessary measure for political clarity before elections. The conclusion emphasizes the importance of upholding the rule of law while ensuring public trust in the judiciary.

I need to structure these points clearly, making sure to note the accusations from the National Rally, historical precedents, expert opinions on judicial independence, the discussion on legal procedures like provisional execution, and the broader implications for democracy and political accountability. Keeping it concise but covering all key aspects without getting lost in details.


**Résumé des points clés du transcript :**

1. **Condamnation de Marine Le Pen** :  
   - Marine Le Pen a été condamnée à 5 ans d’inéligibilité avec exécution immédiate, compromettant sa candidature à la présidentielle de 2027.  
   - Le Rassemblement National (RN) dénonce une justice politisée et accuse les magistrats, notamment la juge Bénédicte de Perthuis, d’ingérence politique. Cette dernière a été placée sous protection policière suite à des menaces.

2. **Contextualisation historique** :  
   - Les accusations de partialité de la justice ne sont pas nouvelles : dans les années 1990, des figures du Parti Socialiste (affaire Urba) ou de droite (François Fillon en 2017, Nicolas Sarkozy dans l’affaire libyenne) ont également critiqué l’institution judiciaire.  
   - Exemples de condamnations de personnalités politiques (Jérôme Cahuzac, Alain Juppé, Patrick Balkany, etc.), montrant que les poursuites touchent tous les bords.

3. **Débat sur l’indépendance et la politisation de la justice** :  
   - **Magalie Lafourcat (magistrate)** : Souligne que les juges sont soumis à une déontologie stricte et contrôlés via des voies de recours (appel, Cour européenne). Elle défend leur indépendance, mais reconnaît que les critiques doivent rester respectueuses.  
   - **Henri Guéno (ancien conseiller de Sarkozy)** : Évoque une possible "ivresse de toute-puissance" chez certains juges et plaide pour un droit de critique encadré, sans menaces.  
   - **Jean Garrigues (historien)** : Relie ces accusations à une contestation plus large des institutions démocratiques et à une "brutalisation" du débat public.

4. **Exécution provisoire et enjeux législatifs** :  
   - La condamnation immédiate de Marine Le Pen repose sur l’exécution provisoire, une procédure légale permettant d’appliquer une peine avant un appel.  
   - Discussions sur une éventuelle réforme de cette règle, notamment portée par le Premier ministre Gabriel Attal, pour éviter des conséquences irréversibles en période électorale.  
   - Débat entre nécessité d’apaiser le climat politique et risques de lois impulsives ("lois d’émotion").

5. **Conséquences politiques et éthiques** :  
   - Les experts soulignent que juger un politique a des répercussions politiques, mais rappellent que la justice applique la loi, non des intentions partisanes.  
   - Appel à la probité des élus et au respect des procédures, malgré les tensions actuelles. La condamnation de Marine Le Pen pour "détournement de fonds publics" est jugée grave, mais soumise à un appel en 2026.

6. **Enjeux démocratiques** :  
   - Le discrédit jeté sur la justice (termes comme "juges rouges" par le RN) est perçu comme dangereux pour l’État de droit.  
   - Importance de distinguer critique légitime et attaques systémiques, tout en préservant l’indépendance judiciaire face aux pressions politiques.

**Conclusion** : Le transcript expose une polarisation croissante autour de la justice, mêlant enjeux juridiques, historiques et politiques. Les débats soulignent la nécessité d’équilibrer indépendance judiciaire, responsabilité des élus et confiance citoyenne, dans un contexte où l’instrumentalisation du droit menace la démocratie.

---
undefined: https://labs.perplexity.ai/?utm_source=copy_output

NOMS CORRIGES


*Résumé des points clés du transcript :*

1. *Condamnation de Marine Le Pen* :  
   - Marine Le Pen a été condamnée à 5 ans d’inéligibilité avec exécution immédiate, compromettant sa candidature à la présidentielle de 2027.  
   - Le Rassemblement National (RN) dénonce une justice politisée et accuse les magistrats, notamment la juge Bénédicte de Perthuis, d’ingérence politique. Cette dernière a été placée sous protection policière suite à des menaces.

2. *Contextualisation historique* :  
   - Les accusations de partialité de la justice ne sont pas nouvelles : dans les années 1990, des figures du Parti Socialiste (affaire Urba) ou de droite (François Fillon en 2017, Nicolas Sarkozy dans l’affaire libyenne) ont également critiqué l’institution judiciaire.  
   - Exemples de condamnations de personnalités politiques (Jérôme Cahuzac, Alain Juppé, Patrick Balkany, etc.), montrant que les poursuites touchent tous les bords.

3. *Débat sur l’indépendance et la politisation de la justice* :  
   - *Magali Lafourcade (magistrate)* : Souligne que les juges sont soumis à une déontologie stricte et contrôlés via des voies de recours (appel, Cour européenne). Elle défend leur indépendance, mais reconnaît que les critiques doivent rester respectueuses.  
   - *Henri Guaino (ancien conseiller de Sarkozy)* : Évoque une possible "ivresse de toute-puissance" chez certains juges et plaide pour un droit de critique encadré, sans menaces.  
   - *Jean Garrigues (historien)* : Relie ces accusations à une contestation plus large des institutions démocratiques et à une "brutalisation" du débat public.

4. *Exécution provisoire et enjeux législatifs* :  
   - La condamnation immédiate de Marine Le Pen repose sur l’exécution provisoire, une procédure légale permettant d’appliquer une peine avant un appel.  
   - Discussions sur une éventuelle réforme de cette règle, notamment portée par le Premier ministre Gabriel Attal, pour éviter des conséquences irréversibles en période électorale.  
   - Débat entre nécessité d’apaiser le climat politique et risques de lois impulsives ("lois d’émotion").

5. *Conséquences politiques et éthiques* :  
   - Les experts soulignent que juger un politique a des répercussions politiques, mais rappellent que la justice applique la loi, non des intentions partisanes.  
   - Appel à la probité des élus et au respect des procédures, malgré les tensions actuelles. La condamnation de Marine Le Pen pour "détournement de fonds publics" est jugée grave, mais soumise à un appel en 2026.

6. *Enjeux démocratiques* :  
   - Le discrédit jeté sur la justice (termes comme "juges rouges" par le RN) est perçu comme dangereux pour l’État de droit.  
   - Importance de distinguer critique légitime et attaques systémiques, tout en préservant l’indépendance judiciaire face aux pressions politiques.

*Conclusion* : Le transcript expose une polarisation croissante autour de la justice, mêlant enjeux juridiques, historiques et politiques. Les débats soulignent la nécessité d’équilibrer indépendance judiciaire, responsabilité des élus et confiance citoyenne, dans un contexte où l’instrumentalisation du droit menace la démocratie.


Okay, I need to summarize the key points from this French transcript. Let me start by reading through it carefully to understand the main arguments and events discussed.

The transcript talks about Marine Le Pen and the Rassemblement National facing legal issues, specifically a conviction for misusing public funds. They're organizing a rally in Paris, drawing parallels to François Fillon's 2017 campaign scandal. The justice system's decision is contested by Le Pen, who claims it's a political attack to undermine her party before elections. The Prime Minister's comments questioning the judicial decision are causing controversy, with accusations of undermining judicial independence. There's a discussion about legal procedures, appeals, and the possibility of reducing Le Pen's ineligibility period. The concept of a "government of judges" is brought up, criticizing judicial overreach. The summary should highlight the political reactions, legal strategies, and implications for Le Pen's political future. Also, mention the comparison to past political figures and the broader accusation of a political conspiracy against the RN. Ensure to note the Prime Minister's stance, the opposition's criticism, and the potential legislative changes regarding provisional execution of sentences.


*Résumé des points clés du transcript :*

1. *Condamnation de Marine Le Pen* :  
   - Marine Le Pen et le Rassemblement National (RN) sont condamnés pour *détournement de fonds publics* liés à des assistants parlementaires fictifs au Parlement européen.  
   - La peine initiale inclut *5 ans d'inéligibilité*, mais une réduction à 18 mois en appel (prévu pour 2026) pourrait permettre à Le Pen de se présenter à la présidentielle.  

2. *Stratégie politique du RN* :  
   - Le Pen dénonce un *"complot politique"* orchestré par des adversaires (mention de Martin Schulz, ex-président du Parlement européen, et de la gauche).  
   - Le RN organise un *grand rassemblement à Paris* (place Vauban), comparé à celui de François Fillon en 2017 pendant l'affaire des emplois fictifs.  
   - Accusations de *"gouvernement des juges"* et de déni démocratique, avec des appels à mobiliser la rue contre la justice.  

3. *Réactions politiques* :  
   - Le Premier ministre *Gabriel Attal* critique l'*exécution provisoire* des peines d'inéligibilité, suggérant une réforme législative pour permettre des recours.  
   - La gauche et des juristes dénoncent une *instrumentalisation de la justice* par le RN et mettent en garde contre une *dérive illibérale*.  
   - Des divisions apparaissent au sein de la majorité présidentielle, notamment avec le Modem, qui défend l'indépendance de la justice.  

4. *Enjeux juridiques* :  
   - Débat sur la *suspension de l'exécution provisoire* de la peine via un recours au premier président de la cour d'appel.  
   - Possibilité d'un *pourvoi en cassation* pour Le Pen, mais la jurisprudence rend complexe la suspension de l'inéligibilité.  

5. *Conséquences politiques* :  
   - Marine Le Pen risque de *faire campagne avec une condamnation*, ce qui affaiblirait sa crédibilité.  
   - Des tensions internes au RN pourraient émerger si la situation persiste, avec des risques de *remise en cause de son leadership*.  

6. *Contexte médiatique et historique* :  
   - Comparaisons avec l'affaire Fillon (2017) et référence à des *théories du complot* pour expliquer les procédures judiciaires.  
   - Le Canard Enchaîné souligne les contradictions juridiques et les risques d'une *loi rétroactive* pour adoucir les peines.  

*Conclusion* : L'affaire cristallise des tensions entre justice et politique, avec le RN accusant une partialité du système, tandis que l'opposition dénonce une stratégie victimisante. L'issue dépendra des recours juridiques et de la mobilisation politique en amont des élections.

---
undefined: https://labs.perplexity.ai/?utm_source=copy_output


Okay, let's tackle this query. The user wants a summary of a French video transcript in French. The title is "Redéfinition pénale du viol, inéligibilité de Le Pen, Grok: Le Club | 28 minutes | ARTE". The description mentions several topics covered in the show, including a new legal definition of rape, reactions to Trump's trade war, debates on Le Pen's ineligibility, a social media trend mocking the French, Elon Musk's AI Grok turning against him, and a segment on Jewish humor.

First, I need to read through the provided transcript carefully. The transcript is a detailed discussion among panelists on various current events. The key points are:

1. *Redéfinition pénale du viol*: The National Assembly voted to include "consent" in the legal definition of rape. Supporters see it as a cultural shift towards protecting victims, especially those in a state of shock. However, there's opposition arguing it might burden victims to prove non-consent.

2. *Inéligibilité de Marine Le Pen*: After her conviction, there's a proposal to remove immediate ineligibility for elected officials, allowing them to remain in office during appeals. Opinions are divided, with some seeing it as undermining judicial authority and others advocating for democratic choice.

3. *Grok AI vs. Elon Musk*: The AI developed by Musk's company criticized him, highlighting issues of free speech and control over AI, drawing parallels to Frankenstein's monster turning against its creator.

4. *Guerre commerciale de Trump*: The segment discusses reactions from UK and Australian leaders to Trump's tariffs, emphasizing the need for a coordinated European response versus individual negotiations.

5. *Tendances sur les réseaux sociaux*: A TikTok trend mocking the French ("I don't wanna be French") led to patriotic counter-memes showcasing French pride in healthcare, gastronomy, etc.

6. *Humour juif*: Interview with Constance La Grange about her book on Jewish humor, emphasizing self-deprecation and its role in coping with adversity, especially relevant amidst rising antisemitism.

7. *Actualité internationale*: Brief mentions of Myanmar's earthquake, Hungary leaving the ICC, and the US election implications.

I need to structure these points concisely in French, ensuring each major topic is covered with its key arguments and outcomes. The user likely wants a clear, structured summary without missing critical debates or perspectives. I should avoid personal opinions and stick to the transcript's content, highlighting the main discussions, differing viewpoints, and conclusions where present. Also, noting the cultural and political implications of each topic as discussed by the panelists.


*Résumé des points clés du transcript :* (Généré par IA)

1. *Redéfinition pénale du viol* :  
   - L'Assemblée nationale a voté l'inscription du *consentement* dans la définition légale du viol, une avancée symbolique et juridique pour mieux protéger les victimes (notamment en état de sidération).  
   - Débats entre partisans (protection accrue) et opposants (crainte d'une charge supplémentaire sur les victimes pour prouver le non-consentement).  
   - La ministre Aurore Bergé y voit un tournant culturel, mais des associations soulignent les lacences persistantes (faible taux de condamnation, manque de moyens judiciaires).

2. *Inéligibilité de Marine Le Pen* :  
   - Après sa condamnation, une proposition de loi portée par Éric Ciotti vise à supprimer l’*inéligibilité immédiate*, permettant à un élu de rester en poste pendant les recours en justice.  
   - Les opinions divergent : certains y voient une atteinte à la moralisation de la vie politique, d’autres un respect du choix démocratique (le peuple jugera).  
   - 64 % des Français sont contre ce changement selon un sondage.

3. *Guerre commerciale de Trump* :  
   - Les réactions contrastées des dirigeants (Royaume-Uni pragmatique, Australie critique) face aux droits de douane annoncés par Trump.  
   - L’Europe prépare une réponse coordonnée, mais des divisions persistent (Italie privilégiant la négociation).  
   - Enjeu de souveraineté économique et craintes d’un isolationnisme américain.

4. *Tensions Chine-Taïwan* :  
   - Les exercices militaires chinois autour de Taïwan relancent les craintes d’un conflit, dans un contexte où Trump entretient une ambiguïté sur le soutien américain.  
   - Analysé comme un prélude à un duel stratégique entre États-Unis et Chine pour la domination technologique et géopolitique.

5. *Réseaux sociaux : « I don’t wanna be French »* :  
   - Une tendance TikTok moquant les Français (stéréotypes gastronomiques, historique) a déclenché une vague de réponses patriotiques (système de santé, patrimoine).  
   - Symbolise une forme de résilience collective et de réappropriation identitaire.

6. *Humour juif avec Constance La Grange* :  
   - Son livre *On peut rire de tout (sauf de sa mère)* explore l’humour juif comme arme contre la bêtise et outil d’autodérision.  
   - Malgré un contexte d’antisémitisme croissant, elle défend un humour universel et rassembleur, loin des clichés communautaires.

7. *Grok, l’IA d’Elon Musk* :  
   - L’intelligence artificielle Grok, conçue pour la « liberté d’expression », critique son créateur (fake news, ego), évoquant le mythe de l’arroseur arrosé.  
   - Débat sur le contrôle des IA et leur indépendance vis-à-vis de leurs créateurs.

8. *Actualité internationale* :  
   - *Hongrie* : Viktor Orbán provoque en accueillant Benjamin Netanyahou et en quittant la Cour pénale internationale (CPI).  
   - *Birmanie* : Séisme meurtrier exacerbant les critiques contre la junte militaire, incompétente face à la crise.  
   - *États-Unis* : Élection d’une juge démocrate en Arizona, signal potentiel de rejet de Trump en vue des midterms.

*Conclusion* : L’émission aborde des enjeux sociétaux (viol, justice), géopolitiques (Trump, Taïwan) et culturels (humour, réseaux sociaux), reflétant les tensions entre progrès législatif, souveraineté populaire et défis technologiques.



---
undefined: https://labs.perplexity.ai/?utm_source=copy_output

In [ ]:
import requests
import json
from dataclasses import dataclass, field

import json
from pathlib import Path

assistant_content = "I need to structure these points concisely in French, ensuring each major topic is covered with its key arguments and outcomes. The user likely wants a clear, structured summary without missing critical debates or perspectives. I should avoid personal opinions and stick to the transcript's content, highlighting the main discussions, differing viewpoints, and conclusions where present. Also, noting the cultural and political implications of each topic as discussed by the panelists."

@dataclass(kw_only=True, frozen=True)
class OpenRouterAiProxy:
    api_key: str = field(repr=False)

    def prompt(self, user_content: str):
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json",
            },
            data=json.dumps(
                {
                    "model": "deepseek/deepseek-chat-v3-0324:free",
                    # "model": "deepseek/deepseek-r1-zero:free",
                    "messages": [
                        {"role": "user", "content": user_content},
                        # {"role": "assistant", "content": assistant_content},
                    ],
                }
            ),
        )
        response_dict = ((json.loads(response.content.decode("utf-8"))))
        return response_dict

api_key = json.loads((Path.home() / Path(".secrets/secrets.json")).read_text())[
    "openrouter.ai"
]["api-key"]
proxy = OpenRouterAiProxy(api_key=api_key)
# response = proxy.prompt("Can you talk about shampoo, please answer in a human readable form")
response = proxy.prompt(prompt)
prompt

In [ ]:
print(response["choices"][0]["message"]["content"].replace(r"\boxed{```text", "")[:-1])

In [ ]:
from IPython.display import Markdown

display(Markdown((response["choices"][0]["message"]["content"])))
# display(
#     Markdown(
#         (response["choices"][0]["message"]["content"].replace(r"\boxed{```text", "")[:-1])
#     )
# )
# display(
#     Markdown(
#         (response["choices"][0]["message"]["reasoning"].replace(r"\boxed{```markdown", "")[:-1])
#     )
# )

In [ ]:
from IPython.display import Markdown
display(Markdown((response["choices"][0]["message"]["content"])))
display(Markdown((response["choices"][0]["message"]["reasoning"])))

In [ ]:
response["choices"][0]["message"]

In [ ]:
print(json.dumps(response, indent=4))

display(Markdown((response["choices"][0]["message"]["reasoning"])))


Example output with deepseek

```json
{
    "id": "gen-1743812116-aInRbMV850awhc5qsgxG",
    "provider": "Targon",
    "model": "deepseek/deepseek-r1-zero",
    "object": "chat.completion",
    "created": 1743812116,
    "choices": [
        {
            "logprobs": null,
            "finish_reason": "stop",
            "native_finish_reason": "stop",
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "\\boxed{\"The question 'What is the meaning of life?' is a deeply philosophical one that varies based on individual beliefs, cultural backgrounds, and personal experiences. Common perspectives to find meaning in life include existentialism (where individual actions and choices create meaning), religious views (such as serving God or achieving spiritual enlightenment, depending on one's faith), biological purpose (survival and reproduction), personal fulfillment (relationships, career, self-improvement, happiness), and hedonism (seeking pleasure and avoiding pain). However, there is no single, universally accepted answer to what the meaning of life is.\"}",
                "refusal": null,
                "reasoning": "To provide a meaningful answer about \"the meaning of life,\" it is important to recognize that this question has been a central theme in philosophy, religion, psychology, and many other fields for centuries. The question \"What is the meaning of life?\" often refers to the purpose or significance of human existence. However, the answer can be highly subjective and can vary widely based on one's personal beliefs, cultural background, philosophical outlook, and life experiences.\n\nHere are some common perspectives on the meaning of life:\n\n1. **Philosophical Perspectives:**\n   - **Existentialism:** Prominent existentialist philosophers like Jean-Paul Sartre, Albert Camus, and Friedrich Nietzsche argue that life intrinsically has no purpose or meaning; individuals must create their own meaning through actions, choices, and values. Sartre famously said, \"Existence precedes essence,\" suggesting that humans first exist and without predetermined purpose, and their \"essence\" is defined only through conscious actions.\n   - **Absurdism:** Albert Camus proposed that the search for meaning in an indifferent universe is fundamentally absurd. The only real philosophical question, according to Camus, is whether to commit suicide; however, he suggests that one should embrace the absurdity of life and live in spite of it.\n   - **Nihilism:** Nihilism is the belief that life has no intrinsic meaning or value. This viewpoint often concludes that life is meaningless rather than attempting to find or create meaning.\n\n2. **Religious Perspectives:**\n   - **Christianity:** For many Christians, the meaning of life is to serve God and to achieve salvation through faith in Jesus Christ. A well-known Bible verse that captures this is from Ecclesiastes 12:13: \"Fear God and keep his commandments, for that is the duty of all mankind.\"\n   - **Hinduism:** In Hinduism, the main goal of life can vary among the four \"Purusharthas\": Dharma (duty/righteousness), Artha (wealth), Kama (desire), and Moksha (liberation from the cycle of rebirth).\n   - **Buddhism:** The meaning of life in Buddhism typically centers on understanding the \"Four Noble Truths\" and following the \"Eightfold Path\" with the aim of achieving Nirvana, which is a state of liberation from suffering and the cycle of rebirth.\n   - **Islam:** For Muslims, life's purpose is to worship Allah and live according to the teachings found in the Quran. One key verse is in the Quran (51:56): \"I did not create jinn and humans except to worship Me.\"\n\n3. **Scientific and Biological Perspectives:**\n   - From a strictly biological perspective, one might argue that the \"meaning\" of life is to survive and reproduce so that genes are passed on to future generations. However, this view is often seen as reductive and does not account for human consciousness and cultural constructs.\n  \n4. **Personal and Subjective Perspectives:**\n   - For many people, the meaning of life can be found in personal fulfillment, such as through relationships, career success, creative endeavors, self-improvement, happiness, and contributing positively to society.\n\n5. **Hedonism:**\n   - This philosophy holds that the meaning of life is primarily about seeking pleasure and avoiding pain. However, \"pleasure\" here might be defined broadly and include both physical pleasures and intellectual, emotional, and spiritual fulfillment.\n\nConclusion: The \"meaning of life\" is a question that might not have a single, universally acceptable answer. It is a deeply personal question that each individual may answer differently based on their own values, experiences, and beliefs. \n\nThe question itself invites us to reflect on what makes life valuable or what goals and purposes make our lives meaningful. Thus, each person may need to seek out and define what gives their life a sense of purpose and fulfillment.\n"
            }
        }
    ],
    "usage": {
        "prompt_tokens": 13,
        "completion_tokens": 910,
        "total_tokens": 923
    }
}
```

In [ ]:
response_dict = ((json.loads(response.content.decode("utf-8"))))
print(json.dumps(response_dict, indent=4))

## Test youtube-extract to fetch metadata

In [ ]:
import pafy
import os 

video_url = "https://www.youtube.com/watch?v=VTmwhwu1JEc"
os.environ["PAFY_BACKEND"] =  "internal"
video = pafy.new(video_url)

In [ ]:
import youtube_extract

In [ ]:
dir(youtube_extract)